### Importamos librerias

In [1]:
import pandas
import numpy
import random
import pygame
from collections import deque
import time

pygame 2.1.0 (SDL 2.0.16, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Definimos variables globales

In [2]:
ancho=720 #Ancho de la Pantalla
alto=480  #Largo de la Pantalla
fps=5      #Fps a los que se mueve el juego
vel = 15 #Velocidad de la serpiente recomendacion dejarlo en divisores de 1 
celdas=5   #Celdas del tablero recomendado para ver 23*23 celdas
thickness = 5 #Circunferencia del cuerpo

black = pygame.Color(0, 0, 0)
white = pygame.Color(255, 255, 255)
red = pygame.Color(255, 0, 0)
green = pygame.Color(0, 255, 0)
blue = pygame.Color(0, 0, 255)

tablero=[] #Matriz del tablero donde |-1 Casilla sin nada |-100 Casilla de borde |-50 Cuerpo de la serpiente |100 Comida de la serpiente
snakeHead=[100, 50] #snakeHead actuual de Snake en forma x,y
snakeBody = [[100, 50],
              [90, 50],
              [80, 50],
              [70, 50]
              ] # Crear una cola vacía para saber donde esta el cuerpo de snake
newSitio=[0,0] #Sitio a donde va a ir Snake en forma x,y
comida=[random.randrange(1, (ancho//10)) * 10,
        random.randrange(1, (alto//10)) * 10] #snakeHead actual de la comida en forma x,y
score=0


comidaExist = False # Booleano si la comida existe
moviendome = False #Actualmente se esta moviendo?

#direccion=["der","izq","aba", "arr"] #Arreglo de las direcciones a 3letras para comodidad
dir="RIGHT" #Direccion actual de snake
change_to=dir

### Funciones del juego

In [3]:
def show_score(game_window, color, font, size):
   
    # creating font object score_font
    score_font = pygame.font.SysFont(font, size)
     
    # create the display surface object
    # score_surface
    score_surface = score_font.render('Score : ' + str(score), True, color)
     
    # create a rectangular object for the
    # text surface object
    score_rect = score_surface.get_rect()
     
    # displaying text
    game_window.blit(score_surface, score_rect)


def gameOver():
    global tablero, snakeHead, snakeBody # Hacemos variables globales para modificar su valor dentro
    snakeBody2 = snakeBody.copy()
    snakeBody2.pop()
    if snakeHead[0] <len(tablero)-2 or tablero[snakeHead[1]][snakeHead[0]+1]!=-50:
        snakeHead=[0,0] #snakeHead actuual de Snake en forma x,y
        newSitio=[0,0] #Sitio a donde va a ir Snake en forma x,y
        comida=[0,0] #snakeHead actual de la comida en forma x,y
        snakeBody = deque() # Crear una cola vacía para saber donde esta el cuerpo de snake
        score=0
    if snakeHead[1] < 0 or snakeHead[1] > alto-10:
        snakeHead=[0,0] #snakeHead actuual de Snake en forma x,y
        newSitio=[0,0] #Sitio a donde va a ir Snake en forma x,y
        comida=[0,0] #snakeHead actual de la comida en forma x,y
        snakeBody = deque() # Crear una cola vacía para saber donde esta el cuerpo de snake
        score=0
    for block in snakeBody2:
        if snakeHead[0] == block[0] and snakeHead[1] == block[1]:
            snakeHead=[0,0] #snakeHead actuual de Snake en forma x,y
            newSitio=[0,0] #Sitio a donde va a ir Snake en forma x,y
            comida=[0,0] #snakeHead actual de la comida en forma x,y
            snakeBody = deque() # Crear una cola vacía para saber donde esta el cuerpo de snake
            score=0
        

def crearSnake():
    global tablero, snakeHead, snakeBody # Hacemos variables globales para modificar su valor dentro
    snakeHead[0] = random.randint(1, celdas) # X del snakeHead donde va a estar la cabeza de snake
    snakeHead[1] = random.randint(1, celdas) # Y del snakeHead donde va a estar la cabeza de snake
    tablero[snakeHead[1]][snakeHead[0]]=0 
    
def creaTablero():
    global tablero, snakeHead, snakeBody # Hacemos variables globales para modificar su valor dentro
    
    for i in range( celdas + 2): #Hacemos una matriz de -1 con borde -100 O(n²)
        temp = []
        if i == 0 or i == celdas + 1:
            for j in range(celdas + 2):
                temp.append( -100 ) 
        else:
            for j in range(celdas + 2):
                if j==0 or j==celdas+1:
                    temp.append( -100 )
                else:
                    temp.append( -1 )
        tablero.append( temp )
    print(tablero)
    crearSnake() #Llamamos a la funcion para crear a snake
    
    snakeBody.append([snakeHead[0],snakeHead[1]]) # Ponemos la posicion de snake en la snakeBody para saber donde esta la cabeza
    return 

def creaComida():
    global comidaExist, comida
    while True: #Repetimos el loop hasta que se pueda crear la comida
        numero_aleatoriox = random.randint(1, celdas) #X
        numero_aleatorioy = random.randint(1, celdas) #Y
        if tablero[numero_aleatorioy][numero_aleatoriox]!=-50: #Verificamos q no exista snake en la possicion generada
            tablero[numero_aleatorioy][numero_aleatoriox]=100 #Creamos la comida en la matriz
            comida[0]=numero_aleatoriox #Guardamos la posicion de X en la comida
            comida[1]=numero_aleatorioy #Guardamos la posicion de Y en la comida
            comidaExist = True
            break

def dibujaCeldas(screen): # Dibujamos las celdas
    color = "black"
    for i in range(celdas):
        temporal=i*alto/celdas
        start_pos = (0, temporal)
        end_pos = (ancho, temporal)
        pygame.draw.line(screen, color, start_pos, end_pos, thickness)

        temporal=i*ancho/celdas
        start_pos = (temporal, 0)
        end_pos = (temporal, alto)
        pygame.draw.line(screen, color, start_pos, end_pos, thickness)
    
def dibujaSnakeyComida(screen): 
    global tablero # Llamamos la variables que se modifica
    mat=[]
    for i in range(celdas+2):
        temp=[]
        if i==0 or i==celdas+1:
            for j in range(celdas+2):
                temp.append(-100) 
        else:
            for j in range(celdas+2):
                if j==0 or j==celdas+1:
                    temp.append(-100)
                else:
                    temp.append(-1)
        mat.append(temp) #LLenamos la matriz
   
    cabeza=snakeHead #Guardamos las cordenadas de la cabeza
    for i in snakeBody: # Pintamos a toda snake de verde
        mat[i[1]][i[0]]=-50
        pygame.draw.circle(screen, "green",( ancho/celdas*(i[0]-1) + ancho/celdas/2 , alto/celdas*(i[1]-1) + alto/celdas/2), 20)
            
    mat[cabeza[1]][cabeza[0]]=0 # Pintamos la cabeza de snake de negro
    pygame.draw.circle(screen, "black",( ancho/celdas*(cabeza[0]-1) + ancho/celdas/2 , alto/celdas*(cabeza[1]-1) + alto/celdas/2), 20)
            
    mat[comida[1]][comida[0]]=100 # Pintamos la comida
    pygame.draw.circle(screen, "red",( ancho/celdas*(comida[0]-1) + ancho/celdas/2 , alto/celdas*(comida[1]-1) + alto/celdas/2), 20)
    tablero=mat

def dibuja(screen):
    global comidaExist
    if not (comidaExist): #Verificamos la existencia de la comida si no existe:
        creaComida() #Creamos la comida
    dibujaCeldas(screen) # Dibujamos celdas
    dibujaSnakeyComida(screen) # Dibujamos a snake y Comida
    
def move(dir): 
    global snakeHead, comidaExist, snakeBody, moviendome, score, comida #, newSitio #Variables a modificar y usar
    x = snakeHead[0] #renombramos para mayor comodidad
    y = snakeHead[1]
    nx =newSitio[0]
    ny =newSitio[1]
    # Nos movemos de acuerdo a la direccion
    if dir == "RIGHT":
        x=x+vel

    if dir == "LEFT":
        x=x-vel
    
    if dir == "DOWN":
        y=y+vel
    
    if dir == "UP":
        y=y-vel
            
    if x == comida[0] and y == comida[1]:
        score+=10
        comidaExist=False
        snakeBody.append([x,y])
    else:
        snakeBody.append([x,y])
        snakeBody.popleft()
         
    # if aDonde == "der": 
    #     if (x>nx): # Cuando llegamos  decimos que ya no nos movemos 
    #         moviendome=False
    #         x=nx
    #         if tablero[y][x]==100: # si es comida agregamos a snake una bolita mas
    #             comidaExist=False
    #             snakeBody.append([x,y])
    #             score+=10
                
    #         else:  # Sino agregamos la bolita de posicion y quitamos la cola
    #             snakeBody.append([x,y])
    #             snakeBody.popleft()

            
    snakeHead[0] = x 
    snakeHead[1] = y
    # newSitio[0] = nx 
    # newSitio[1] = ny


In [4]:
def init():
    creaTablero()
    pygame.init()
    screen = pygame.display.set_mode((ancho, alto))
    clock = pygame.time.Clock()
    running = True
    change_to='der'
    direction = 'der'
    while running:
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    change_to = 'UP'
                if event.key == pygame.K_DOWN:
                    change_to = 'DOWN'
                if event.key == pygame.K_LEFT:
                    change_to = 'LEFT'
                if event.key == pygame.K_RIGHT:
                    change_to = 'RIGHT'
            if event.type == pygame.QUIT:
                running = False
        
        if change_to == 'UP' and direction != 'DOWN':
            direction = 'UP'
        if change_to == 'DOWN' and direction != 'UP':
            direction = 'DOWN'
        if change_to == 'LEFT' and direction != 'RIGHT':
            direction = 'LEFT'
        if change_to == 'RIGHT' and direction != 'LEFT':
            direction = 'RIGHT'

        # fill the screen with a color to wipe away anything from last frame
        screen.fill((255, 253, 208))

        # RENDER YOUR GAME HERE
        dibuja(screen)
        move(direction)
        show_score(screen, white, 'times new roman', 20)
        gameOver()
        #print('-----')
        #print('head: ', sitio)
        #print('fila: ', fila)

        # Definir el grosor de la línea (en este caso, 5 píxeles) 

        # Dibujar la línea en la ventana

        # flip() the display to put your work on screen
        pygame.display.flip()

        clock.tick(fps)  # limits FPS to 60

    pygame.quit()

In [5]:
#init()

In [6]:
try:
    init()
except:
    pygame.quit()
     
    # quit the program
    quit()

[[-100, -100, -100, -100, -100, -100, -100], [-100, -1, -1, -1, -1, -1, -100], [-100, -1, -1, -1, -1, -1, -100], [-100, -1, -1, -1, -1, -1, -100], [-100, -1, -1, -1, -1, -1, -100], [-100, -1, -1, -1, -1, -1, -100], [-100, -100, -100, -100, -100, -100, -100]]


: 

In [1]:
# importing libraries
import pygame
import time
import random

snake_speed = 15

# Window size
window_x = 720
window_y = 480

# defining colors
black = pygame.Color(0, 0, 0)
white = pygame.Color(255, 255, 255)
red = pygame.Color(255, 0, 0)
green = pygame.Color(0, 255, 0)
blue = pygame.Color(0, 0, 255)

try:
    # Initialising pygame
    pygame.init()

    # Initialise game window
    pygame.display.set_caption('GeeksforGeeks Snakes')
    game_window = pygame.display.set_mode((window_x, window_y))

    # FPS (frames per second) controller
    fps = pygame.time.Clock()

    # defining snake default position
    snake_position = [100, 50]

    # defining first 4 blocks of snake body
    snake_body = [[100, 50],
                [90, 50],
                [80, 50],
                [70, 50]
                ]
    # fruit position
    fruit_position = [random.randrange(1, (window_x//10)) * 10,
                    random.randrange(1, (window_y//10)) * 10]

    fruit_spawn = True

    # setting default snake direction towards
    # right
    direction = 'RIGHT'
    change_to = direction

    # initial score
    score = 0

    # displaying Score function
    def show_score(choice, color, font, size):

        # creating font object score_font
        score_font = pygame.font.SysFont(font, size)
        
        # create the display surface object
        # score_surface
        score_surface = score_font.render('Score : ' + str(score), True, color)
        
        # create a rectangular object for the text
        # surface object
        score_rect = score_surface.get_rect()
        
        # displaying text
        game_window.blit(score_surface, score_rect)

    # game over function
    def game_over():

        # creating font object my_font
        my_font = pygame.font.SysFont('times new roman', 50)
        
        # creating a text surface on which text
        # will be drawn
        game_over_surface = my_font.render(
            'Your Score is : ' + str(score), True, red)
        
        # create a rectangular object for the text
        # surface object
        game_over_rect = game_over_surface.get_rect()
        
        # setting position of the text
        game_over_rect.midtop = (window_x/2, window_y/4)
        
        # blit will draw the text on screen
        game_window.blit(game_over_surface, game_over_rect)
        pygame.display.flip()
        
        # after 2 seconds we will quit the program
        time.sleep(2)
        
        # deactivating pygame library
        pygame.quit()
        
        # quit the program
        quit()


    # Main Function
    while True:
        
        # handling key events
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    change_to = 'UP'
                if event.key == pygame.K_DOWN:
                    change_to = 'DOWN'
                if event.key == pygame.K_LEFT:
                    change_to = 'LEFT'
                if event.key == pygame.K_RIGHT:
                    change_to = 'RIGHT'

        # If two keys pressed simultaneously
        # we don't want snake to move into two
        # directions simultaneously
        if change_to == 'UP' and direction != 'DOWN':
            direction = 'UP'
        if change_to == 'DOWN' and direction != 'UP':
            direction = 'DOWN'
        if change_to == 'LEFT' and direction != 'RIGHT':
            direction = 'LEFT'
        if change_to == 'RIGHT' and direction != 'LEFT':
            direction = 'RIGHT'

        # Moving the snake
        if direction == 'UP':
            snake_position[1] -= 10
        if direction == 'DOWN':
            snake_position[1] += 10
        if direction == 'LEFT':
            snake_position[0] -= 10
        if direction == 'RIGHT':
            snake_position[0] += 10

        # Snake body growing mechanism
        # if fruits and snakes collide then scores
        # will be incremented by 10
        snake_body.insert(0, list(snake_position))
        if snake_position[0] == fruit_position[0] and snake_position[1] == fruit_position[1]:
            score += 10
            fruit_spawn = False
        else:
            snake_body.pop()
            
        if not fruit_spawn:
            fruit_position = [random.randrange(1, (window_x//10)) * 10,
                            random.randrange(1, (window_y//10)) * 10]
            
        fruit_spawn = True
        game_window.fill(black)
        
        for pos in snake_body:
            pygame.draw.rect(game_window, green,
                            pygame.Rect(pos[0], pos[1], 10, 10))
        pygame.draw.rect(game_window, white, pygame.Rect(
            fruit_position[0], fruit_position[1], 10, 10))

        # Game Over conditions
        if snake_position[0] < 0 or snake_position[0] > window_x-10:
            game_over()
        if snake_position[1] < 0 or snake_position[1] > window_y-10:
            game_over()

        # Touching the snake body
        for block in snake_body[1:]:
            if snake_position[0] == block[0] and snake_position[1] == block[1]:
                game_over()

        # displaying score continuously
        show_score(1, white, 'times new roman', 20)

        # Refresh game screen
        pygame.display.update()

        # Frame Per Second /Refresh Rate
        fps.tick(snake_speed)
except:
    print('gameover')

pygame 2.1.0 (SDL 2.0.16, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
gameover


: 